In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options, service=service)

driver.get('https://www.margxt.fr/le-pokedex-de-paldea-liste-des-pokemon-et-ou-les-trouver-dans-pokemon-ecarlate-et-violet/')

In [3]:
table = driver.find_element(By.TAG_NAME, 'table').find_element(By.TAG_NAME, 'tbody')

pokedex = []
for e in table.find_elements(By.TAG_NAME, 'tr'):
    poke = e.text.split('\n')[:2]
    poke[1] = poke[1].split(' ')[0]
    if poke[0].isnumeric():
        img = e.find_element(By.TAG_NAME, 'img').get_attribute('src')
        poke.append(img)
        pokedex.append(tuple(poke))

In [4]:
# Missing from table elements
pokedex[50] = ('51', 'Mustébouée', 'https://www.margxt.fr/wp-content/uploads/2022/02/Muste%CC%81boue%CC%81e.png')
pokedex[354] = ('355', 'Polarhume', 'https://www.margxt.fr/wp-content/uploads/2022/10/Polarhume.png')

In [5]:
df = pd.DataFrame(pokedex, columns=['num', 'name', 'image'])
df.to_csv('Pokédex de Paldea.csv', index=False)

In [6]:
_pokedex = [f'({x[0]})<br>{x[1]}<br><img decoding="async" loading="lazy" width="50" height="50" style="width: 50px;" src="{x[2]}" alt="">' for x in pokedex]

In [7]:
n = 30
pokedex_chunks = [_pokedex[i:i + n] for i in range(0, len(_pokedex), n)]

In [8]:
html_body = ''
html_body += '''<!doctype html>
<html lang="en">
  <head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@4.0.0/dist/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
  </head>
  <style type="text/css">
  table td:hover {
  background-color: rgba(2, 123, 255);
  }
  td {
  text-align: center;
  }
  th {
  background-color: #228082;

  text-align: center;
  }
  table {
      background-image: linear-gradient(to left bottom, #2ed8cc, #18cbcb, #09bfc9, #10b2c4, #1fa5be);
  }
  </style>

'''

In [9]:
for idx, chunk in enumerate(pokedex_chunks, start=1):
    n = 6
    l2 = [chunk[i:i + n] for i in range(0, len(chunk), n)]
    df = pd.DataFrame(l2)
    df.index += 1 
    df.columns += 1
    html_body += f'<h4 align="center">Boîte {idx}</h4>\n'
    html_body += df.to_html()

In [10]:
html_body += '''
    <!-- Optional JavaScript -->
    <!-- jQuery first, then Popper.js, then Bootstrap JS -->
    <script src="https://code.jquery.com/jquery-3.2.1.slim.min.js" integrity="sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN" crossorigin="anonymous"></script>
    <script src="https://cdn.jsdelivr.net/npm/popper.js@1.12.9/dist/umd/popper.min.js" integrity="sha384-ApNbgh9B+Y1QKtv3Rn7W3mgPxhU9K/ScQsAP7hUibX39j7fakFPskvXusvfa0b4Q" crossorigin="anonymous"></script>
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@4.0.0/dist/js/bootstrap.min.js" integrity="sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl" crossorigin="anonymous"></script>
  </body>
  </html>
'''

html_body = html_body.replace('class="dataframe"', 'class="table table-dark table-hover"').replace('&lt;', '<').replace('&gt;', '>').replace('None', '')


In [12]:
with open('Pokédex de Paldea.html', 'w') as f:
    f.write(html_body)